In [43]:
import cv2 as cv
import numpy as np

In [69]:
imagem_original = cv.imread('EU.JPG')
#imagem_original = cv.resize(imagem_original,(768,1024))
imagem_cinza = cv.cvtColor(imagem_original,cv.COLOR_BGR2GRAY)
points = []
size = imagem_original.shape
rect = (0,0,size[1],size[0])

In [70]:
def rect_contains(rect, point) :
    if point[0] < rect[0] :
        return False
    elif point[1] < rect[1] :
        return False
    elif point[0] > rect[2] :
        return False
    elif point[1] > rect[3] :
        return False
    return True   

def draw_cricle(event,x,y,flags,param):
    global points
    if event == cv.EVENT_LBUTTONDBLCLK:
        cv.circle(imagem_original,(x,y),1,(255,0,0),-1)
        cv.imshow('ROI',imagem_original)
        mousex,mousey = x,y
        points.append((mousex,mousey))
        
def draw_triangles(img,subdiv): #Delaunay Triangulation
    
    lst_triangulos = subdiv.getTriangleList()
    
    for tri in lst_triangulos:

        pt1 = (tri[0],tri[1])
        pt2 = (tri[2],tri[3])
        pt3 = (tri[4],tri[5])

        if rect_contains(rect,pt1) and rect_contains(rect,pt2) and rect_contains(rect,pt3):

            cv.line(img,pt1,pt2,(32,201,66),1,cv.LINE_AA,0)
            cv.line(img,pt2,pt3,(32,201,66),1,cv.LINE_AA,0)
            cv.line(img,pt3,pt1,(32,201,66),1,cv.LINE_AA,0)

def draw_voronoi(img, subdiv) :

    ( facets, centers) = subdiv.getVoronoiFacetList([])

    for i in range(0,len(facets)) :
        ifacet_arr = []
        for f in facets[i] :
            ifacet_arr.append(f)

        ifacet = np.array(ifacet_arr, np.int)
        color = (np.random.randint(0, 255), np.random.randint(0, 255), np.random.randint(0, 255))
        cv.fillConvexPoly(img, ifacet, color, cv.LINE_AA, 0);
        ifacets = np.array([ifacet])
        cv.polylines(img, ifacets, True, (0, 0, 0), 1, cv.LINE_AA, 0)
        #cv.circle(img, (centers[i][0], centers[i][1]), 3, (0, 0, 0), cv.FILLED, cv.LINE_AA, 0)
        
def desenha_manual():
    subdiv  = cv.Subdiv2D(rect)
    global points
    cv.namedWindow('ROI')
    cv.setMouseCallback('ROI',draw_cricle)
    while True:
        cv.imshow('ROI',imagem_original)
        key = cv.waitKey()  
        if key == ord('q'):
            cv.destroyWindow('ROI')
            break
    for p in points:
        subdiv.insert(p)
    
    return subdiv
    
def desenha_automatico(treshX,treshY):
    subdiv  = cv.Subdiv2D(rect)
    imagem_canny = cv.Canny(imagem_cinza,treshX,treshY)
    contours,_ = cv.findContours(imagem_canny,cv.RETR_LIST,cv.CHAIN_APPROX_SIMPLE)
    for contorno in contours:
        x,y = contorno[0][0]
        subdiv.insert((x,y))
    return subdiv,imagem_canny

In [54]:
subdiv = desenha_manual()

In [71]:
subdiv,imagem_canny = desenha_automatico(60,60)

In [72]:
imagem_triangulos = np.full(size,(0,0,0),np.uint8)
imagem_vitral = np.full(size,(0,0,0),np.uint8)

draw_triangles(imagem_triangulos,subdiv)
draw_voronoi(imagem_vitral,subdiv)

cv.imshow('Triangles',imagem_triangulos)
cv.imshow('Vitral',imagem_vitral)
cv.imshow('Canny',imagem_canny)
cv.waitKey()
cv.destroyAllWindows()

C:\Users\Heron\AppData\Local\Temp/ipykernel_3868/1723950869.py:32: DeprecationWarning: an integer is required (got type numpy.float32).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  cv.line(img,pt1,pt2,(32,201,66),1,cv.LINE_AA,0)
C:\Users\Heron\AppData\Local\Temp/ipykernel_3868/1723950869.py:33: DeprecationWarning: an integer is required (got type numpy.float32).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  cv.line(img,pt2,pt3,(32,201,66),1,cv.LINE_AA,0)
C:\Users\Heron\AppData\Local\Temp/ipykernel_3868/1723950869.py:34: DeprecationWarning: an integer is required (got type numpy.float32).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  cv.line(img,pt3,pt1,(32,201,66),1,cv.LINE_AA,0)
C:\Users\Heron\AppData\Local\Temp/ipykernel_3868/1723950869.py:45: DeprecationWarning: `np.int` is a depre

In [67]:
cv.imwrite('EU_tri.jpg',imagem_triangulos)
cv.imwrite('EU_vitral.jpg',imagem_vitral)
#cv.imwrite('amanda_canny.jpg',imagem_canny)

True